## 打开EXCEL文件

In [206]:
import xlwings as xw
import pandas as pd
import warnings
import numpy as np
import pyodbc
from sqlalchemy import create_engine
from tqdm import tqdm_notebook as tqdm  
warnings.filterwarnings("ignore")

## 1.同比、环比

In [207]:
######################定义需要的参数###########################################
文件路径 = r"C:\Users\hp\Desktop\1.csv"
Sheet名 = '1'
表格范围 = 'B1:AF85954'
统计日期列名 = '统计日期'
商品ID列名 = '商品ID'
列名列表 = ['毛利额', '补偿后毛利额']  # 这里列出您想计算的所有列
写入起始位置 = 'AH1'
##############################################################################
# 打开 Excel 工作簿
wb = xw.Book(文件路径)
sheet = wb.sheets[Sheet名]
# 从 Excel 中读取数据
data = sheet.range(表格范围).options(pd.DataFrame, index=False, header=1).value
# 按照统计日期和商品ID进行排序
data = data.sort_values(by=[统计日期列名, 商品ID列名])
data[统计日期列名] = pd.to_datetime(data[统计日期列名])
data.set_index([统计日期列名, 商品ID列名], inplace=True)
def calculate_and_save_growth_rates(data, 列名列表, sheet, 写入起始位置):
    # 对每个列名进行计算
    for 列名 in 列名列表:
        data[f'{列名}同比'] = data.groupby(商品ID列名)[列名].pct_change(periods=12).round(2)
        data[f'{列名}环比'] = data.groupby(商品ID列名)[列名].pct_change(periods=1).round(2)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.reset_index(inplace=True)
    # 将计算结果写回 Excel 文件
    columns_to_write = [f'{列名}同比' for 列名 in 列名列表] + [f'{列名}环比' for 列名 in 列名列表]
    sheet.range(写入起始位置).options(index=False).value = data[columns_to_write]
    wb.save()
# 调用该函数进行计算并保存结果
calculate_and_save_growth_rates(data, 列名列表, sheet, 写入起始位置)

## 上传数据库

In [208]:
def create_and_insert_data(table_name, df, server, database):
    # 使用sqlalchemy创建连接
    connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
    engine = create_engine(connection_string)
    
    # 如果表存在，则删除
    if engine.has_table(table_name):
        with engine.connect() as conn:
            conn.execute(f"DROP TABLE {table_name}")
    
    # 使用pandas的to_sql方法插入数据，利用chunksize和tqdm显示进度条
    chunk_size = 500  # 可以根据你的需求调整
    num_chunks = (len(df) // chunk_size) + 1
    for i in tqdm(range(num_chunks), desc="插入数据"):
        df.iloc[i*chunk_size:(i+1)*chunk_size].to_sql(name=table_name, con=engine, if_exists='append', index=False)
create_and_insert_data(table_name='table_rate', df=data, server='localhost', database='data')

插入数据:   0%|          | 0/172 [00:00<?, ?it/s]